<a href="https://colab.research.google.com/github/arpitpatelsitapur/my-py-torch-journey/blob/main/optuna_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 11.6 MB/s eta 0:00:00


In [2]:
# Import necessary libraries
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the Pima Indian Diabetes dataset from sklearn
# Note: Scikit-learn's built-in 'load_diabetes' is a regression dataset.
# We will load the actual diabetes dataset from an external source
import pandas as pd

# Load the Pima Indian Diabetes dataset (from UCI repository)
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI',
           'DiabetesPedigreeFunction', 'Age', 'Outcome']

# Load the dataset
df = pd.read_csv(url, names=columns)

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
import numpy as np

# Replace zero values with NaN in columns where zero is not a valid value
cols_with_missing_vals = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[cols_with_missing_vals] = df[cols_with_missing_vals].replace(0, np.nan)

# Impute the missing values with the mean of the respective column
df.fillna(df.mean(), inplace=True)

# Check if there are any remaining missing values
print(df.isnull().sum())


Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [4]:
# Split into features (X) and target (y)
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Split data into training and test sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Optional: Scale the data for better model performance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Check the shape of the data
print(f'Training set shape: {X_train.shape}')
print(f'Test set shape: {X_test.shape}')


Training set shape: (537, 8)
Test set shape: (231, 8)


In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [6]:
# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters


[I 2025-09-19 08:28:06,837] A new study created in memory with name: no-name-55f56b8e-92ba-4560-821f-0b613a9e839f
[I 2025-09-19 08:28:10,035] Trial 0 finished with value: 0.7616387337057727 and parameters: {'n_estimators': 193, 'max_depth': 6}. Best is trial 0 with value: 0.7616387337057727.
[I 2025-09-19 08:28:11,007] Trial 1 finished with value: 0.7746741154562384 and parameters: {'n_estimators': 145, 'max_depth': 17}. Best is trial 1 with value: 0.7746741154562384.
[I 2025-09-19 08:28:11,354] Trial 2 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 57, 'max_depth': 9}. Best is trial 1 with value: 0.7746741154562384.
[I 2025-09-19 08:28:12,055] Trial 3 finished with value: 0.7616387337057727 and parameters: {'n_estimators': 123, 'max_depth': 9}. Best is trial 1 with value: 0.7746741154562384.
[I 2025-09-19 08:28:12,902] Trial 4 finished with value: 0.7746741154562384 and parameters: {'n_estimators': 126, 'max_depth': 14}. Best is trial 1 with value: 0.77467411

In [7]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7802607076350093
Best hyperparameters: {'n_estimators': 125, 'max_depth': 19}


In [8]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


## Samplers in Optuna

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [10]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters

[I 2025-09-19 09:52:09,614] A new study created in memory with name: no-name-69c5bb5b-8ec2-48e2-8e71-8db98a8d585a
[I 2025-09-19 09:52:10,620] Trial 0 finished with value: 0.7597765363128491 and parameters: {'n_estimators': 195, 'max_depth': 4}. Best is trial 0 with value: 0.7597765363128491.
[I 2025-09-19 09:52:11,741] Trial 1 finished with value: 0.7821229050279329 and parameters: {'n_estimators': 197, 'max_depth': 19}. Best is trial 1 with value: 0.7821229050279329.
[I 2025-09-19 09:52:12,279] Trial 2 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 94, 'max_depth': 7}. Best is trial 1 with value: 0.7821229050279329.
[I 2025-09-19 09:52:12,811] Trial 3 finished with value: 0.7579143389199254 and parameters: {'n_estimators': 98, 'max_depth': 6}. Best is trial 1 with value: 0.7821229050279329.
[I 2025-09-19 09:52:13,763] Trial 4 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 156, 'max_depth': 19}. Best is trial 1 with value: 0.782122905

In [11]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7821229050279329
Best hyperparameters: {'n_estimators': 197, 'max_depth': 19}


In [12]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


In [13]:
search_space = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [5, 10, 15, 20]
}

In [14]:
# Create a study and optimize it using GridSampler
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.GridSampler(search_space))
study.optimize(objective)

[I 2025-09-19 10:01:00,189] A new study created in memory with name: no-name-e525c3de-20be-4ab4-b554-d0dae72dd7b2
[I 2025-09-19 10:01:00,747] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 5}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-09-19 10:01:01,599] Trial 1 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 150, 'max_depth': 10}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-09-19 10:01:01,912] Trial 2 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 50, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-09-19 10:01:02,490] Trial 3 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 100, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-09-19 10:01:03,090] Trial 4 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 20}. Best is trial 2 with value: 0.772811

In [15]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7746741154562384
Best hyperparameters: {'n_estimators': 50, 'max_depth': 5}


In [16]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


## Optuna Visualizations

In [17]:
# For visualizations
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice, plot_contour, plot_param_importances

In [18]:
# 1. Optimization History
plot_optimization_history(study).show()

In [19]:
# 2. Parallel Coordinates Plot
plot_parallel_coordinate(study).show()

In [20]:
# 3. Slice Plot
plot_slice(study).show()

In [21]:
# 4. Contour Plot
plot_contour(study).show()

In [22]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

## Optimizing Multiple ML Models

In [23]:
# Importing the required libraries
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [24]:
# Define the objective function for Optuna
def objective(trial):
    # Choose the algorithm to tune
    classifier_name = trial.suggest_categorical('classifier', ['SVM', 'RandomForest', 'GradientBoosting'])

    if classifier_name == 'SVM':
        # SVM hyperparameters
        c = trial.suggest_float('C', 0.1, 100, log=True)
        kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
        gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

        model = SVC(C=c, kernel=kernel, gamma=gamma, random_state=42)

    elif classifier_name == 'RandomForest':
        # Random Forest hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])

        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            bootstrap=bootstrap,
            random_state=42
        )

    elif classifier_name == 'GradientBoosting':
        # Gradient Boosting hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

        model = GradientBoostingClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )

    # Perform cross-validation and return the mean accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()
    return score

In [25]:
# Create a study and optimize it using CmaEsSampler
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2025-09-19 10:03:44,191] A new study created in memory with name: no-name-a447b5e7-937b-491f-9262-538a54c941d8
[I 2025-09-19 10:03:44,231] Trial 0 finished with value: 0.7653631284916201 and parameters: {'classifier': 'SVM', 'C': 6.364261108236312, 'kernel': 'rbf', 'gamma': 'auto'}. Best is trial 0 with value: 0.7653631284916201.
[I 2025-09-19 10:03:44,266] Trial 1 finished with value: 0.7486033519553073 and parameters: {'classifier': 'SVM', 'C': 7.846266349307165, 'kernel': 'rbf', 'gamma': 'auto'}. Best is trial 0 with value: 0.7653631284916201.
[I 2025-09-19 10:03:44,313] Trial 2 finished with value: 0.7299813780260708 and parameters: {'classifier': 'SVM', 'C': 39.41180674460542, 'kernel': 'rbf', 'gamma': 'scale'}. Best is trial 0 with value: 0.7653631284916201.
[I 2025-09-19 10:03:45,507] Trial 3 finished with value: 0.7392923649906891 and parameters: {'classifier': 'GradientBoosting', 'n_estimators': 52, 'learning_rate': 0.0594805356028857, 'max_depth': 16, 'min_samples_split': 

In [26]:
# Retrieve the best trial
best_trial = study.best_trial
print("Best trial parameters:", best_trial.params)
print("Best trial accuracy:", best_trial.value)

Best trial parameters: {'classifier': 'SVM', 'C': 0.1428248877678663, 'kernel': 'linear', 'gamma': 'scale'}
Best trial accuracy: 0.7895716945996275


In [27]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_C,params_bootstrap,params_classifier,params_gamma,params_kernel,params_learning_rate,params_max_depth,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.765363,2025-09-19 10:03:44.194017,2025-09-19 10:03:44.231181,0 days 00:00:00.037164,6.364261,NaN,SVM,auto,rbf,NaN,NaN,NaN,NaN,NaN,COMPLETE
1,1,0.748603,2025-09-19 10:03:44.232082,2025-09-19 10:03:44.266800,0 days 00:00:00.034718,7.846266,NaN,SVM,auto,rbf,NaN,NaN,NaN,NaN,NaN,COMPLETE
2,2,0.729981,2025-09-19 10:03:44.267805,2025-09-19 10:03:44.313342,0 days 00:00:00.045537,39.411807,NaN,SVM,scale,rbf,NaN,NaN,NaN,NaN,NaN,COMPLETE
3,3,0.739292,2025-09-19 10:03:44.314383,2025-09-19 10:03:45.507433,0 days 00:00:01.193050,NaN,NaN,GradientBoosting,NaN,NaN,0.059481,16.0,1.0,10.0,52.0,COMPLETE
4,4,0.733706,2025-09-19 10:03:45.508432,2025-09-19 10:03:47.933354,0 days 00:00:02.424922,NaN,NaN,GradientBoosting,NaN,NaN,0.184177,11.0,5.0,2.0,152.0,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.785847,2025-09-19 10:04:25.891386,2025-09-19 10:04:26.111516,0 days 00:00:00.220130,62.128164,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
96,96,0.783985,2025-09-19 10:04:26.112483,2025-09-19 10:04:26.154846,0 days 00:00:00.042363,1.383132,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
97,97,0.761639,2025-09-19 10:04:26.155767,2025-09-19 10:04:26.212350,0 days 00:00:00.056583,0.172980,NaN,SVM,auto,rbf,NaN,NaN,NaN,NaN,NaN,COMPLETE
98,98,0.754190,2025-09-19 10:04:26.213218,2025-09-19 10:04:27.931707,0 days 00:00:01.718489,NaN,NaN,GradientBoosting,NaN,NaN,0.127187,3.0,4.0,6.0,300.0,COMPLETE


In [28]:
study.trials_dataframe()['params_classifier'].value_counts()

,count
params_classifier,
SVM,79
GradientBoosting,11
RandomForest,10


In [29]:
study.trials_dataframe().groupby('params_classifier')['value'].mean()

,value
params_classifier,
GradientBoosting,0.742509
RandomForest,0.767412
SVM,0.774792


In [30]:
# 1. Optimization History
plot_optimization_history(study).show()

In [31]:
# 3. Slice Plot
plot_slice(study).show()

In [32]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

In [34]:
!pip install optuna-integration[xgboost]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 3.7 MB/s eta 0:00:00


In [35]:
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import numpy as np

# Load the Iris dataset
X, y = load_iris(return_X_y=True)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function for XGBoost
def objective(trial):
    # Hyperparameter search space
    param = {
        'verbosity': 0,
        'objective': 'multi:softprob',
        'num_class': 3,
        'eval_metric': 'mlogloss',  # Ensure that the eval_metric is specified here
        'booster': 'gbtree',
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'n_estimators': 300,
    }

    # Create DMatrix for XGBoost
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    # Define a pruning callback based on evaluation metrics
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "eval-mlogloss")  # Match the metric name in the evals list

    # Train the model
    bst = xgb.train(
        param,
        dtrain,
        num_boost_round=300,
        evals=[(dtrain, "train"), (dtest, "eval")],  # Ensure the eval datasets and names are specified
        early_stopping_rounds=30,
        callbacks=[pruning_callback]
    )

    # Predict on the test set
    preds = bst.predict(dtest)
    best_preds = [int(np.argmax(line)) for line in preds]

    # Return accuracy as the objective value
    accuracy = accuracy_score(y_test, best_preds)
    return accuracy

# Create a study with pruning
study = optuna.create_study(direction='maximize', pruner=optuna.pruners.SuccessiveHalvingPruner())
study.optimize(objective, n_trials=50)

# Output the best trial
print(f"Best trial: {study.best_trial.params}")
print(f"Best accuracy: {study.best_value}")


[I 2025-09-19 10:09:59,393] A new study created in memory with name: no-name-4e0db1db-335e-4a62-93e5-1fe25156c609


[0]	train-mlogloss:1.00700	eval-mlogloss:1.01185
[1]	train-mlogloss:0.89194	eval-mlogloss:0.88746
[2]	train-mlogloss:0.78133	eval-mlogloss:0.77008
[3]	train-mlogloss:0.68685	eval-mlogloss:0.66958
[4]	train-mlogloss:0.60823	eval-mlogloss:0.58814
[5]	train-mlogloss:0.53863	eval-mlogloss:0.51239
[6]	train-mlogloss:0.48229	eval-mlogloss:0.45471
[7]	train-mlogloss:0.43606	eval-mlogloss:0.40662
[8]	train-mlogloss:0.39562	eval-mlogloss:0.36235
[9]	train-mlogloss:0.36327	eval-mlogloss:0.32848
[10]	train-mlogloss:0.33299	eval-mlogloss:0.29331
[11]	train-mlogloss:0.30656	eval-mlogloss:0.26205
[12]	train-mlogloss:0.29362	eval-mlogloss:0.24774
[13]	train-mlogloss:0.27220	eval-mlogloss:0.22241
[14]	train-mlogloss:0.25963	eval-mlogloss:0.20865
[15]	train-mlogloss:0.24781	eval-mlogloss:0.19425
[16]	train-mlogloss:0.23712	eval-mlogloss:0.18258
[17]	train-mlogloss:0.22846	eval-mlogloss:0.17356
[18]	train-mlogloss:0.21786	eval-mlogloss:0.16179
[19]	train-mlogloss:0.21632	eval-mlogloss:0.15948
[20]	train

[I 2025-09-19 10:10:01,013] Trial 0 finished with value: 1.0 and parameters: {'lambda': 9.928543593120486e-08, 'alpha': 0.0004247064861629639, 'eta': 0.12123662941111783, 'gamma': 0.09424647478740729, 'max_depth': 4, 'min_child_weight': 5, 'subsample': 0.550504861766044, 'colsample_bytree': 0.5067515219897217}. Best is trial 0 with value: 1.0.


[0]	train-mlogloss:1.05137	eval-mlogloss:1.04368
[1]	train-mlogloss:0.99468	eval-mlogloss:0.98277
[2]	train-mlogloss:0.93065	eval-mlogloss:0.91768
[3]	train-mlogloss:0.86880	eval-mlogloss:0.85242
[4]	train-mlogloss:0.81468	eval-mlogloss:0.79475
[5]	train-mlogloss:0.76205	eval-mlogloss:0.73801
[6]	train-mlogloss:0.71625	eval-mlogloss:0.69100
[7]	train-mlogloss:0.67890	eval-mlogloss:0.65639
[8]	train-mlogloss:0.64329	eval-mlogloss:0.61776
[9]	train-mlogloss:0.61651	eval-mlogloss:0.59051
[10]	train-mlogloss:0.58457	eval-mlogloss:0.55423
[11]	train-mlogloss:0.55499	eval-mlogloss:0.52170
[12]	train-mlogloss:0.53792	eval-mlogloss:0.50493
[13]	train-mlogloss:0.51265	eval-mlogloss:0.47640
[14]	train-mlogloss:0.49744	eval-mlogloss:0.45909
[15]	train-mlogloss:0.47677	eval-mlogloss:0.43807
[16]	train-mlogloss:0.46239	eval-mlogloss:0.42150
[17]	train-mlogloss:0.44709	eval-mlogloss:0.40590
[18]	train-mlogloss:0.43104	eval-mlogloss:0.38873
[19]	train-mlogloss:0.41835	eval-mlogloss:0.37396
[20]	train

[I 2025-09-19 10:10:02,393] Trial 1 finished with value: 1.0 and parameters: {'lambda': 1.2820399620942102e-08, 'alpha': 1.9960424213699455e-05, 'eta': 0.06068904589801694, 'gamma': 0.0013124863088096474, 'max_depth': 7, 'min_child_weight': 8, 'subsample': 0.5729119707260314, 'colsample_bytree': 0.6074052688376279}. Best is trial 0 with value: 1.0.


...

[0]	train-mlogloss:1.00569	eval-mlogloss:1.00583
[1]	train-mlogloss:0.90074	eval-mlogloss:0.89892


[I 2025-09-19 10:10:30,909] Trial 48 pruned. Trial was pruned at iteration 2.


[0]	train-mlogloss:0.87579	eval-mlogloss:0.86710
[1]	train-mlogloss:0.65236	eval-mlogloss:0.63428
[2]	train-mlogloss:0.50093	eval-mlogloss:0.47559


[I 2025-09-19 10:10:30,943] Trial 49 pruned. Trial was pruned at iteration 2.


Best trial: {'lambda': 9.928543593120486e-08, 'alpha': 0.0004247064861629639, 'eta': 0.12123662941111783, 'gamma': 0.09424647478740729, 'max_depth': 4, 'min_child_weight': 5, 'subsample': 0.550504861766044, 'colsample_bytree': 0.5067515219897217}
Best accuracy: 1.0


In [36]:
from optuna.visualization import plot_intermediate_values

# 1. Plot intermediate values during the trials
plot_intermediate_values(study).show()